# QEを用意する

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt

# Google Colaboratory上でQuantum ESPRESSO(QE)を用意する

まず自分のgoogle driveをマウントする。（google accountが必要です。）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ASEがインストールされていなければ使えるようにする。

In [ ]:
if importlib.util.find_spec("ase") is None:
    !pip install git+https://github.com/minoru-otani/ase.git@qe_rism

QEのソースを取ってくる。現時点で最新のQE-7.0rc1にRISMが導入されたものを使う。

In [ ]:
!git clone -b qe-7.0-rism https://github.com/minoru-otani/q-e.git

コンパイルする。

In [ ]:
!apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc
%cd q-e/
!DFLAGS='-D__FFTW3 -D__MPI' FFT_LIBS='-lfftw3' ./configure
!make pw pp

既にコンパイルしたQEがある場合にはコピーする。ない場合にはダウンロードしてコンパイルすると同時に、次回に向けてMy Driveにzipファイルを退避させておく。

In [ ]:
%cd /content
!zip -r /content/drive/'My Drive'/qe-7.0-rism.zip q-e

環境を確認する。

In [ ]:
!cat /proc/cpuinfo # 2cpu, 1core for 1cpuなので２プロセス走らせる

ASE用にパスを設定

In [ ]:
os.environ['PATH'] = "/content/q-e/bin:"+os.environ['PATH']
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun --allow-run-as-root -np 2 pw.x -in PREFIX.pwi > PREFIX.pwo"
qef_pseudo = 'http://www.quantum-espresso.org/wp-content/uploads/upf_files/'
pseudo_dir = "/content/q-e/pseudo/"

ESM/RISM_exampleを動かすためにenvironment_variablesを編集する

In [ ]:
%cd /content/q-e/
!cat -n environment_variables #どの行を編集するか見る→７６行目をコメントアウトして、７７行目を生かせば良いことが分かる。

In [ ]:
%%bash
sed -i -e "76s/PARA_PREFIX/#PARA_PREFIX/" environment_variables
sed -i -e "77d" environment_variables
sed -i -e "77i PARA_PREFIX=\"mpirun --allow-run-as-root -np 2\"" environment_variables
cat -n environment_variables

ESM_exampleを動かしてみる。

In [ ]:
%cd /content/q-e/PW/examples/ESM_example
!./run_example

計算結果を見てみる。

In [ ]:
%cd results
!cat H2O_pbc.out

In [ ]:
%cd results
!cat Al001_bc1.out

ESM1をプロットしてみる。

In [1]:
def plot_esm1(filename,titlename=''):
    esm1 = np.loadtxt(filename,comments='#')

    fig = plt.figure(
        figsize   = (6, 4), # inch
        dpi       = 100,    # dpi
        edgecolor = 'black',
        linewidth = '1'
    )

    fig.subplots_adjust(wspace=0.5, hspace=0.5)
    fig.suptitle(titlename) 

    
    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(223)
    ax4 = fig.add_subplot(224)

    ax1.set_xlabel('z (A)')
    ax1.set_ylabel('rho (e/A)')
    ax2.set_xlabel('z (A)')
    ax2.set_ylabel('V_hartree (eV)')
    ax3.set_xlabel('z (A)')
    ax3.set_ylabel('V_ion (eV)')
    ax4.set_xlabel('z (A)')
    ax4.set_ylabel('V_electrostatic (eV)')
    
    ax4.axhline(0.0, linewidth='1', linestyle='dashed', color='black')

    ax1.plot(esm1[:,0], esm1[:,1], color='black',linestyle='solid')
    ax2.plot(esm1[:,0], esm1[:,2], color='black',linestyle='solid')
    ax3.plot(esm1[:,0], esm1[:,3], color='black',linestyle='solid')
    ax4.plot(esm1[:,0], esm1[:,4], color='black',linestyle='solid')

    plt.show()

In [ ]:
plot_esm1('H2O_bc1.esm1', titlename='H2O bc1')

RISM_exampleを動かしてみる。

In [ ]:
%cd /content/q-e/PW/examples/RISM_example
!./run_example